In [1]:
import os

In [2]:
print('parent process:', os.getpid())

parent process: 5710


In [3]:
import numpy as np

In [4]:
import numcodecs
numcodecs.__version__

'0.2.1'

In [6]:
compressor = numcodecs.Blosc()
compressor

Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)

In [7]:
data = np.arange(1000000)
enc = compressor.encode(data)

In [8]:
def worker(task):
    pid = os.getpid()
    print('[%s:%s] begin' % (task, pid))
    dec = np.frombuffer(compressor.decode(enc), dtype=data.dtype)[task:task+10]
    print('[%s:%s] decoded %s' % (task, pid, dec))

In [9]:
worker(0)

[0:5710] begin
[0:5710] decoded [0 1 2 3 4 5 6 7 8 9]


In [10]:
import multiprocessing
pool = multiprocessing.Pool(processes=5)
pool.map(worker, range(5))
pool.close()
pool.join()

[3:5741] begin
[2:5740] begin
[1:5739] begin
[4:5742] begin
[0:5738] begin


KeyboardInterrupt: 

In [11]:
numcodecs.blosc.use_threads = False

In [12]:
pool = multiprocessing.Pool(processes=5)
pool.map(worker, range(5))
pool.close()
pool.join()

[4:5820] begin
[2:5818] begin
[0:5816] begin
[3:5819] begin
[1:5817] begin
[1:5817] decoded [ 1  2  3  4  5  6  7  8  9 10]
[3:5819] decoded [ 3  4  5  6  7  8  9 10 11 12]
[0:5816] decoded [0 1 2 3 4 5 6 7 8 9]
[2:5818] decoded [ 2  3  4  5  6  7  8  9 10 11]
[4:5820] decoded [ 4  5  6  7  8  9 10 11 12 13]
